In [1]:
import json
from openai import AzureOpenAI

deployment = "gpt-4o"
client = AzureOpenAI()


def get_current_cluster_state(cluster_name):
    print(f"cluster:{cluster_name}")
    return  """ERROR: Failed to pull image "chaocai/docker/dsp:latest"""

funcs = {"get_current_cluster_state": get_current_cluster_state}


def run_conversation(msg):
    response = client.chat.completions.create(
        model=deployment,
        messages=msg,
        functions=[
            {
                "name": "get_current_cluster_state",
                "description": "Get the current state in a given cluster",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "cluster_name": {
                            "type": "string",
                            "description": "the name of the cluster"
                        },
                    },
                    "required": ["cluster_name"],
                }
            }
        ],
        function_call="auto",
    )
    
    message = response.choices[0].message
    print("----- message ----")
    print(message)
    print("----- message ----")
    
    if not message.function_call:
        return message
    
    function_name = message.function_call.name
    function_args = json.loads(message.function_call.arguments)
    print("function_name: ", function_name, "\nfunction_args: ", function_args)
    
    # res = funcs[function_name](function_args["cluster_name"])
    res = funcs[function_name](**function_args)
    
    message.content = None
    msg.append(message)
    msg.append({
        "role": "function",
        "name": function_name,
        "content": res,
    })
    
    return run_conversation(msg)

def run(inputs):
    msg=[{"role":"user","content":inputs}]
    res = run_conversation(msg)
    return res.content



In [2]:
result = run("Hello")
print(result)

----- message ----
ChatCompletionMessage(content='Hi there! How can I assist you today?', role='assistant', function_call=None, tool_calls=None)
----- message ----
Hi there! How can I assist you today?


In [3]:
result = run("What's wrong with the cluster 'DSP'? And if there's an error, give me some suggestion. ")
print(result)

----- message ----
ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"cluster_name":"DSP"}', name='get_current_cluster_state'), tool_calls=None)
----- message ----
function_name:  get_current_cluster_state 
function_args:  {'cluster_name': 'DSP'}
cluster:DSP
----- message ----
ChatCompletionMessage(content='The error indicates that the cluster \'DSP\' failed to pull the Docker image "chaocai/docker/dsp:latest". Here are a few suggestions to resolve the issue:\n\n1. **Check Image Repository**:\n   - Ensure that the Docker image exists in the repository.\n   - Verify that the repository URL is correct and that the tag "latest" exists.\n\n2. **Verify Network Connectivity**:\n   - Confirm that the cluster has internet access to pull images from the repository.\n   - Check for any proxy or firewall configurations that might be blocking the request.\n\n3. **Authentication**:\n   - If the image repository requires authentication, ensure that the clus